In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.porter import PorterStemmer
import re
import pickle
from sklearn.feature_extraction import DictVectorizer
import gensim
from gensim.models import Word2Vec
import time
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn import preprocessing
from sklearn import metrics
import seaborn as sns

stopwords = set(ENGLISH_STOP_WORDS)
stopwords.add("tomorrow")
stopwords.add("day")
stopwords.add("http")
stopwords.add("3rd")
stopwords.add("u002c")
stopwords.add("time")
stopwords.add("1st")
stopwords.add("today")
stopwords.add("make")
stopwords.add("think")
stopwords.add("u2019m")
stopwords.add("saturday")
stopwords.add("tuesday")
stopwords.add("wednesday")
stopwords.add("people")
stopwords.add("did")
stopwords.add("will")
stopwords.add("said")
stopwords.add("say")
stopwords.add("says")
stopwords.add("it")
stopwords.add("they")
stopwords.add("are")
stopwords.add("that")
stopwords.add("saying")

In [2]:
# ------------------ REMOVE SPECIAL CHARS AND STOPWORDS from train
trainData = pd.read_csv('train2017.tsv', sep="\t", encoding = "utf-8")

stemmed = []
removed = []  # cleaned tweets
tokens = []
similar_words = []
vect = []
tokenized_tweet = []

for i in range(0, trainData.shape[0]):
	removed.append(re.sub(r"[^a-zA-Z0-9]+", ' ', trainData.loc[i][3]))

	word_tokens = word_tokenize(removed[i])
	filtered_sentence = []
	for w in word_tokens: 
		if w.lower() not in stopwords: 
			filtered_sentence.append(w.lower())

			# STEM
			porter = PorterStemmer()
			stemmed.extend([porter.stem(w.lower())]) 
	tokens.append(filtered_sentence)
	removed[i] = ' '.join(filtered_sentence)

In [3]:
# ------------------ REMOVE SPECIAL CHARS AND STOPWORDS from test
testData = pd.read_csv('test2017.tsv', sep="\t", encoding = "utf-8")

stemmed2 = []
removed2 = []  # cleaned tweets
tokens2 = []
similar_words2 = []
vect2 = []
tokenized_tweet2 = []

for i in range(0, testData.shape[0]):
	removed2.append(re.sub(r"[^a-zA-Z0-9]+", ' ', testData.loc[i][3]))

	word_tokens2 = word_tokenize(removed2[i])
	filtered_sentence2 = []
	for w in word_tokens2: 
		if w.lower() not in stopwords: 
			filtered_sentence2.append(w.lower())

			# STEM
			porter2 = PorterStemmer()
			stemmed2.extend([porter2.stem(w.lower())]) 
	tokens2.append(filtered_sentence2)
	removed2[i] = ' '.join(filtered_sentence2)

In [4]:
# CREATE WORD EMBEDDINGS

# ------------------ w2v train ------------------

model_w2v = gensim.models.Word2Vec(
            tokens,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2,
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
            seed = 34)

model_w2v.train(tokens, total_examples= len(removed), epochs=20)

model_w2v.save('embeddings.pkl')

In [5]:
# LOAD WORD EMBEDDINGS

new_model = gensim.models.Word2Vec.load('embeddings.pkl')

In [6]:
# AFFIN LEXICA

affin = pd.read_csv('affin.txt', sep="\t", encoding = "utf-8")

affin_lexica = []
for i in range(0, affin.shape[0]):
	vals = {}
	vals['word'] = affin.loc[i][0]
	vals['vector'] = affin.loc[i][1]
	affin_lexica.append(vals)

In [7]:
# TWEET VECTORS

# ------------------ w2v train ------------------

vec = []
vec_tweet = []   # list with the vectors of each tweet

for i in range(0, trainData.shape[0]):
	vec = np.zeros(201)  # pos 201 for lexica
	wcount = len(removed[i].split())
	for word in removed[i].split():
		for j in range(0,200):
			if word in new_model.wv.vocab:
				vec[j] += (new_model[word][j] / wcount)
		if word in affin_lexica:
			vec[200] = affin_lexica[word]
	vec_tweet.append(vec)

# print (vec_tweet[0])

# EXPORT TWEET VECTORS TO FILE
tv_output = open("tweetvectors.pkl", "wb")
pickle.dump(vec_tweet, tv_output)
tv_output.close()

/home/athina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


[-1.65066015e-01 -1.01045091e-01  1.14212856e-01  5.77932309e-02
 -6.96794328e-02  1.10225257e-01 -2.03491599e-01 -3.38839713e-02
 -2.75364742e-01  2.30094700e-01  3.83810033e-01  1.66269127e-01
 -2.11221636e-01  1.85103175e-01  4.87432075e-02 -1.14479540e-01
 -7.34377195e-02 -8.99893567e-02  1.29985578e-01  3.85854447e-01
 -1.68923436e-01 -1.57861213e-01 -2.57856622e-01  1.61487778e-02
  1.97000790e-02  5.84129178e-02 -5.44269140e-01 -1.16109610e-01
  8.58304368e-02  1.56476113e-01 -6.68982911e-02  2.25978569e-01
  1.60397035e-01  2.10748809e-01 -1.15914147e-01 -2.83041410e-03
 -1.89363313e-02  2.64590528e-01 -1.65608092e-01 -3.59845604e-02
  1.22831022e-01  3.37625882e-01 -2.13334709e-01  1.71807877e-02
 -1.57130579e-01  8.22499549e-02  1.44180584e-01  4.50663308e-02
  1.29118836e-01 -6.73709634e-02 -1.98930364e-02 -1.22538634e-01
 -1.50742502e-02  3.62732334e-01  9.53227588e-03 -8.06608157e-03
 -2.44499243e-01 -1.24881596e-01 -1.26082501e-01  2.24337077e-01
  2.88640742e-01 -5.15868

In [8]:
# LOAD TWEET VECTORS for train

tv_file = open("tweetvectors.pkl", "rb")
vec_tweet = pickle.load(tv_file)
tv_file.close()

In [9]:
# TWEET VECTORS

# ------------------ w2v test ------------------

vec2 = []
vec_tweet_test = []    # list with the vectors of each tweet

for i in range(0, testData.shape[0]):
	vec2 = np.zeros(201)
	wcount = len(removed2[i].split())
	for word in removed2[i].split():
		for j in range(0,200):
			if word in new_model.wv.vocab:
				vec2[j] += (new_model[word][j] / wcount)
		if word in affin_lexica:
			vec2[200] = affin_lexica[word]
	vec_tweet_test.append(vec2)

# EXPORT TWEET VECTORS TO FILE
tv2_output = open("tweetvectorstest.pkl", "wb")
pickle.dump(vec_tweet_test, tv2_output)
tv2_output.close()

/home/athina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [10]:
# LOAD TWEET VECTORS for test

tv2_file = open("tweetvectorstest.pkl", "rb")
vec_tweet_test = pickle.load(tv2_file)
tv2_file.close()

In [11]:
# BAG OF WORDS

# ------------------ train ------------------

bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000, stop_words='english') 
bow_xtrain = bow_vectorizer.fit_transform(removed)

# ------------------ test ------------------

bow_vectorizer2 = CountVectorizer(max_df=1.0, min_df=1, max_features=1000, stop_words='english') 
bow_xtrain2 = bow_vectorizer2.fit_transform(removed2)

In [12]:
# EXPORT BAG TO FILE

# ------------------ train ------------------

bow_output = open("bagofwords.pkl", "wb")
pickle.dump(bow_xtrain, bow_output)
bow_output.close()

# ------------------ test ------------------

bow_output2 = open("bagofwords_test.pkl", "wb")
pickle.dump(bow_xtrain2, bow_output2)
bow_output2.close()

In [13]:
# LOAD BAG

# ------------------ train ------------------

bow_file = open("bagofwords.pkl", "rb")
bag = pickle.load(bow_file)
bow_file.close()

bow_xtrain = bag
features = bow_vectorizer.get_feature_names()
bag = bag.toarray()
bow_df = pd.DataFrame(bag, columns = features)

bow_xtrain = bow_xtrain.toarray()

# ------------------ test ------------------

bow_file2 = open("bagofwords_test.pkl", "rb")
bag2 = pickle.load(bow_file2)
bow_file2.close()

bow_xtrain_test = bag2
features2 = bow_vectorizer2.get_feature_names()
bag2 = bag2.toarray()
bow_df2 = pd.DataFrame(bag2, columns = features2)

bow_xtrain_test = bow_xtrain_test.toarray()

In [14]:
# TF-IDF

# ------------------ train ------------------

s = []
for i in range(0,trainData.shape[0]):
	s.extend([removed[i]])

tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, max_features=1000, stop_words='english') 
tfidf = tfidf_vectorizer.fit_transform(s)
tfidf = tfidf.toarray()
features = tfidf_vectorizer.get_feature_names()
tfidf_df = pd.DataFrame(np.round(tfidf, 3), columns = features)

# ------------------ test ------------------

s2 = []
for i in range(0,testData.shape[0]):
	s2.extend([removed2[i]])

tfidf_vectorizer2 = TfidfVectorizer(max_df=1.0, min_df=1, max_features=1000, stop_words='english') 
tfidf2 = tfidf_vectorizer2.fit_transform(s2)
tfidf2 = tfidf2.toarray()
features2 = tfidf_vectorizer2.get_feature_names()
tfidf_df2 = pd.DataFrame(np.round(tfidf2, 3), columns = features2)

In [15]:
# EXPORT TF_IDF TO FILE

# ------------------ train ------------------

tfidf_output = open("tfidf.pkl", "wb")
pickle.dump(tfidf_df, tfidf_output)
tfidf_output.close()

# ------------------ test ------------------

tfidf_output2 = open("tfidf_test.pkl", "wb")
pickle.dump(tfidf_df2, tfidf_output2)
tfidf_output2.close()

In [16]:
# LOAD TF_IDF

# ------------------ train ------------------

tfidf_file = open("tfidf.pkl", "rb")
tf = pickle.load(tfidf_file)
tfidf_file.close()

# ------------------ test ------------------

tfidf_file2 = open("tfidf_test.pkl", "rb")
tf_test = pickle.load(tfidf_file2)
tfidf_file2.close()

In [17]:
# READ LABELS

labels = []
for i in range(0, trainData.shape[0]):
	labels.append(trainData.loc[i][2])

testLabels = pd.read_csv('SemEval2017_task4_subtaskA_test_english_gold.txt', sep="\t", encoding = "utf-8")
labels_test = []
for i in range(0, testLabels.shape[0]):
	labels_test.append(testLabels.loc[i][1])

In [18]:
# KNN W2V ONLY FOR TRAIN (train_test_split)

X_train, X_test, y_train, y_test = train_test_split(vec_tweet, labels, test_size=0.4, random_state=4)

In [19]:
# KNN W2V FOR TEST

X_train = vec_tweet
y_train = labels
X_test = vec_tweet_test
y_test = labels_test

In [20]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
# make predictions on the testing set
y_pred = logreg.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

0.5145322803875275


In [21]:
# Repeat for KNN with K=5:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

0.47260441260278435


In [22]:
# SVM W2V ONLY FOR TRAIN (train_test_split)

X_train, X_test, y_train, y_test = train_test_split(vec_tweet, labels, random_state=42, test_size=0.2) #input for this method is any array of features

In [23]:
# SVM W2V FOR TEST

X_train = vec_tweet
y_train = labels
X_test = vec_tweet_test
y_test = labels_test

In [24]:
svc = svm.SVC(kernel='linear', C=1, probability=True)
svc = svc.fit(X_train, y_train)
prediction = svc.predict(X_test) #predict on the validation set
print (f1_score(y_test, prediction, average='micro')) #evaluate on the validation set

0.5126597736709273


In [25]:
# KNN BAG-OF-WORDS ONLY FOR TRAIN (train_test_split)

X_train, X_test, y_train, y_test = train_test_split(bow_xtrain, labels, test_size=0.4, random_state=4)

In [26]:
# KNN BAG-OF-WORDS FOR TEST

X_train = bow_xtrain
y_train = labels
X_test = bow_xtrain_test
y_test = labels_test

In [27]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
# make predictions on the testing set
y_pred = logreg.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

0.393552063828055


In [28]:
# Repeat for KNN with K=5:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
# print (y_pred)
print(metrics.accuracy_score(y_test, y_pred))

['neutral' 'negative' 'neutral' ... 'neutral' 'neutral' 'neutral']
0.3998208906618904


In [29]:
# SVM BAG-OF-WORDS ONLY FOR TRAIN (train_test_split)

X_train, X_test, y_train, y_test = train_test_split(bow_xtrain, labels, random_state=42, test_size=0.2) #input for this method is any array of features

In [30]:
# SVM BAG-OF-WORDS FOR TEST

X_train = bow_xtrain
y_train = labels
X_test = bow_xtrain_test
y_test = labels_test

In [ ]:
svc = svm.SVC(kernel='linear', C=1, probability=True)
svc = svc.fit(X_train, y_train)
prediction = svc.predict(X_test) #predict on the validation set
print (f1_score(y_test, prediction, average='micro')) #evaluate on the validation set

In [ ]:
# KNN TF-IDF ONLY FOR TRAIN (train_test_split)

X_train, X_test, y_train, y_test = train_test_split(tf, labels, test_size=0.4, random_state=4)

In [ ]:
# KNN TF-IDF FOR TEST

X_train = tf
y_train = labels
X_test = tf_test
y_test = labels_test

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
# make predictions on the testing set
y_pred = logreg.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Repeat for KNN with K=5:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
# print (y_pred)
print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
# SVM TF-IDF ONLY FOR TRAIN (train_test_split)

X_train, X_test, y_train, y_test = train_test_split(tf, labels, random_state=42, test_size=0.2) #input for this method is any array of features

In [ ]:
# SVM TF-IDF FOR TEST

X_train = tf
y_train = labels
X_test = tf_test
y_test = labels_test

In [ ]:
svc = svm.SVC(kernel='linear', C=1, probability=True)
svc = svc.fit(X_train, y_train)
prediction = svc.predict(X_test) #predict on the validation set
print (f1_score(y_test, prediction, average='micro')) #evaluate on the validation set